In [1]:
%load_ext autoreload
%autoreload 2

In [8]:
import numpy as np
import matplotlib.pyplot as plt
import time
import girth
# from girth import rasch_conditional
from irt.data.rasch import generate_data, generate_data_positive_scores
from irt.algorithms.spectral_estimator import spectral_estimate
from irt.algorithms import spectral_estimator
from irt.algorithms import conditional_mle
from irt.algorithms import rasch_mml
from irt.algorithms import eigen_vector_method
from irt.evaluation import eval_utils
from irt.evaluation.eval_utils import log_likelihood_heldout, bayesian_auc, pairwise_disagreement_error, top_k_accuracy
from irt.data import data_loader

# # Assume its dichotomous data with True -> 1 and False -> 0
# tagged_data = tag_missing_data(my_data, [0, 1])

In [3]:
def relative_betas_error(beta, betah):
    beta_norm = beta - np.mean(beta)
    betah_norm = betah - np.mean(betah)
    return np.linalg.norm(beta_norm - betah_norm)/np.linalg.norm(beta_norm)

def relative_z_error(z, zh):
    return np.linalg.norm(z - zh)/np.linalg.norm(z)

    

In [15]:
# A = data_loader.lsat()
cutoff = 25

A_lsat = data_loader.lsat()
A_ml_100k, ml_100k_ratings = data_loader.ml_100k(cutoff=cutoff)
A_hetrec_2k, hetrec_2k_ratings  = data_loader.hetrec_2k(cutoff=cutoff)
A_ml_1m, ml_1m_ratings = data_loader.ml_1m(cutoff=cutoff)

print(f"{A_ml_100k.shape}")
print(f"{A_hetrec_2k.shape}")
print(f"{A_ml_1m.shape}")


(1050, 610)
(4735, 2113)
(2934, 6040)


In [16]:
A = A_ml_1m
ratings = ml_1m_ratings
sorted_ratings = sorted(ratings, key=lambda x: x[1], reverse=True)
true_rank = [item for (item, _, _) in sorted_ratings] # Sort from most popular items


In [17]:
sorted_ratings[:100]

[(2152, 4.608695652173913, 69),
 (1445, 4.560509554140127, 628),
 (256, 4.554557700942973, 2227),
 (624, 4.524966261808367, 2223),
 (567, 4.52054794520548, 657),
 (47, 4.517106001121705, 1783),
 (439, 4.510416666666667, 2304),
 (825, 4.507936507936508, 882),
 (667, 4.491489361702127, 470),
 (856, 4.477724741447892, 2514),
 (649, 4.476190476190476, 1050),
 (839, 4.473913043478261, 230),
 (208, 4.453694416583082, 2991),
 (870, 4.452083333333333, 480),
 (571, 4.4498902706656915, 1367),
 (2467, 4.444444444444445, 27),
 (554, 4.426940639269406, 438),
 (865, 4.425646551724138, 928),
 (2539, 4.415607985480944, 551),
 (657, 4.412822049131217, 1669),
 (527, 4.410714285714286, 56),
 (2045, 4.406262708418057, 2459),
 (2245, 4.404651162790698, 215),
 (526, 4.404255319148936, 47),
 (862, 4.401925391095066, 831),
 (658, 4.395973154362416, 1043),
 (852, 4.390371229698376, 1724),
 (668, 4.388888888888889, 1116),
 (2449, 4.387453874538745, 271),
 (906, 4.386993603411514, 938),
 (653, 4.38403041825095, 

In [6]:


errors_arr = []
errors_ase_arr = []
errors_cmle_arr = []
errors_mmle_arr = []
errors_choppin_arr = []
errors_garner_arr = []
errors_saaty_arr = []

time_arr = []
time_ase_arr = []
time_cmle_arr = []
time_mmle_arr = []
time_choppin_arr = []
time_garner_arr = []
time_saaty_arr = []

K_array = [10, 20, 50]
n_array = [50, 100, 500, 1000] #, 2500, 5000]
n_trials = 100
m = 100

expected_num_problems = 20

p = expected_num_problems/m

betas = np.random.normal(0, 2, size=(m,))
true_rank = np.argsort(betas)

for n in n_array:    
    thetas = np.random.normal(0, 4, size=(n,))

    auc_ase = 0.
    auc_cmle = 0.
    auc_mmle = 0.
    loglik_ase = 0.
    loglik_cmle = 0.
    loglik_mmle = 0.
    pd_ase = 0.
    pd_cmle = 0.
    pd_mmle = 0.
    
    time_ase = 0.
    time_cmle = 0.
    time_mmle = 0.
    top_K_ase = []
    top_K_mmle = []
    
    for _ in range(n_trials):
        
        # Generate data
        data = generate_data(betas, thetas, p)
        
        # Partition data
        data, test_data = eval_utils.partition_data(data)
        
        # Conditional MLE
        # start = time.time()
        # est_cmle = conditional_mle.rasch_conditional(data, return_beta=True)
        # time_cmle += 1./n_trials * (time.time() - start)
        # loglik_cmle += 1./n_trials * log_likelihood_heldout(est_cmle, test_data)
        lambd = 0.1
        # Marginal MLE
        start = time.time()
        # est_mmle = rasch_mml.rasch_mml(data, return_beta=True)
        est_mmle = spectral_estimate(data, lambd=lambd, regularization="zero")
        time_mmle += 1./n_trials * (time.time() - start)
        start = time.time()
        loglik_mmle += 1./n_trials * log_likelihood_heldout(est_mmle, test_data)
        auc_mmle += 1./n_trials * bayesian_auc(est_mmle, test_data)
        est_rank_mmle = np.argsort(est_mmle)
        pd_mmle += 1./n_trials * pairwise_disagreement_error(true_rank, est_rank_mmle)
        top_K_mmle += [[top_k_accuracy(true_rank, est_rank_mmle, k) for k in K_array]]

        # Accelerated spectral method
        start = time.time()
        # lambd = 1.
        # lambd = 0.001
        est_ase = spectral_estimate(data, lambd=lambd, regularization="minimal")
        time_ase += 1./n_trials * (time.time() - start)
        loglik_ase += 1./n_trials * log_likelihood_heldout(est_ase, test_data)
        auc_ase += 1./n_trials * bayesian_auc(est_ase, test_data)
        est_rank_ase = np.argsort(est_ase)
        pd_ase += 1./n_trials * pairwise_disagreement_error(true_rank, est_rank_ase)
        top_K_ase += [[top_k_accuracy(true_rank, est_rank_ase, k) for k in K_array]]


    # errors_ase_arr.append(error_ase)
    # errors_cmle_arr.append(error_cmle)
    # errors_mmle_arr.append(error_mmle)

    time_ase_arr.append(time_ase)
    time_cmle_arr.append(time_cmle)
    time_mmle_arr.append(time_mmle)
    top_K_ase = np.array(top_K_ase)
    top_K_mmle = np.array(top_K_mmle)

    
    print(
        f"n={n},m={m}\n" +
        f"Loglik: ASE={loglik_ase}, CMLE={loglik_cmle} MMLE={loglik_mmle}, \n" +
        f"AUC: ASE={auc_ase}, CMLE={auc_cmle}, MMLE={auc_mmle}, \n" +
        f"Rank: ASE={pd_ase}, CMLE={pd_cmle}, MMLE={pd_mmle}, \n" +
        f"Top-K: ASE={np.mean(top_K_ase, 0)}, MMLE={np.mean(top_K_mmle, 0)}\n" +
        f"Time: ASE={time_ase}, CMLE={time_cmle}, MMLE={time_mmle}"
         )

n=50,m=100
Loglik: ASE=-0.6824974940514105, CMLE=0.0 MMLE=-0.6798955685810093, 
AUC: ASE=0.6546390428156521, CMLE=0.0, MMLE=0.6548333222449337, 
Rank: ASE=0.19753333333333328, CMLE=0.0, MMLE=0.197270707070707, 
Top-K: ASE=[0.497  0.6785 0.8064], MMLE=[0.499 0.679 0.806]
Time: ASE=0.012423970699310305, CMLE=0.0, MMLE=0.011484913825988768
n=100,m=100
Loglik: ASE=-0.6588571160785041, CMLE=0.0 MMLE=-0.657690675207806, 
AUC: ASE=0.6756379363384247, CMLE=0.0, MMLE=0.6755379206606327, 
Rank: ASE=0.14279797979797978, CMLE=0.0, MMLE=0.14270707070707073, 
Top-K: ASE=[0.573  0.7485 0.8554], MMLE=[0.57   0.749  0.8564]
Time: ASE=0.008918426036834718, CMLE=0.0, MMLE=0.009145073890686035
n=500,m=100
Loglik: ASE=-0.6726300018296353, CMLE=0.0 MMLE=-0.6721077297773458, 
AUC: ASE=0.6768716071103585, CMLE=0.0, MMLE=0.6768979550061681, 
Rank: ASE=0.07297777777777778, CMLE=0.0, MMLE=0.07288888888888889, 
Top-K: ASE=[0.763 0.895 0.919], MMLE=[0.763  0.896  0.9188]
Time: ASE=0.009128308296203612, CMLE=0.0, M